In [1]:
import cv2
import os
import csv
import numpy as np
import shutil
from skimage.feature import hog
from skimage.morphology import skeletonize
from skimage import exposure

import numpy as np
import tensorflow as tf
from keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# Folder paths
annotation_csv = 'annotations.csv'
dataset_folder = 'dataset'  # Folder containing original images

# Alpha
alpha = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

# Load annotations
with open(annotation_csv, 'r') as file:
    reader = csv.reader(file)
    annotations = list(reader)[1:]  # Skip header


In [3]:
flag = annotations[0][0]
c = 0
image_list1, image_list2, image_list3, image_list4, image_list5 = [], [], [], [], []
label_list = []

# Feature extraction steps
for annotation in annotations:
    c += 1
    image_name, letter, center_x, center_y, dist_x, dist_y = annotation

    # Load original image
    # image_path = os.path.join(dataset_folder, image_name)
    img = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)
    image_name = image_name[image_name.find('\\')+1:image_name.find('.')]
    # img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"Error: {image_name}_{c} not found!")
        continue

    # Crop the letter using the annotation box
    start_x = int(float(center_x) - float(dist_x) / 2)
    start_y = int(float(center_y) - float(dist_y) / 2)
    end_x = int(float(center_x) + float(dist_x) / 2)
    end_y = int(float(center_y) + float(dist_y) / 2)
    cropped = img[start_y:end_y, start_x:end_x]

    # 1. Image Enhancement & Normalization (Histogram Equalization)
    resized = cv2.resize(cropped, (64, 64))
    enhanced = cv2.equalizeHist(resized)
    norm_image = cv2.normalize(enhanced, None, 0, 255, cv2.NORM_MINMAX)
    image_list1.append(norm_image)
    label_list.append(letter)

    # 2. Segmentation (Thresholding)
    _, segmented = cv2.threshold(norm_image, 30, 255, cv2.THRESH_BINARY_INV)
    image_list2.append(segmented)

    # 3. Edge Detection (Canny)
    edges = cv2.Canny(segmented, 100, 200)
    image_list3.append(edges)

    # 4. Skeletonization
    binary = segmented / 255  # Convert to binary (0, 1)
    skeleton = skeletonize(binary).astype(np.uint8) * 255
    image_list4.append(skeleton)

    # 5. HOG (Histogram of Oriented Gradients)
    hog_features, hog_image = hog(segmented, pixels_per_cell=(4, 4), cells_per_block=(2, 2), visualize=True)
    image_list5.append(hog_image)


In [6]:
label_list

['a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'H',
 'H'

In [8]:
indexes = [i for i in range(len(label_list))]
xrl, xtl = train_test_split(indexes, test_size=0.2, random_state=42)

In [ ]:
def lettersSplit(imgList, labelList, uniqueLabels=alpha):
    labelToIdx = {label: idx for idx, label in enumerate(uniqueLabels)}
    numericalLabel = np.array([labelToIdx[label] for label in labelList])

    # Split data into training and testing sets
    x_train, x_test, y_train, y_test = [imgList[i] for i in xrl], [imgList[i] for i in xtl], [numericalLabel[i] for i in xrl], [numericalLabel[i] for i in xtl]

    # Normalize the image data (scale pixel values to range [0, 1])
    x_train = np.array(x_train) / 255.0
    x_test = np.array(x_test) / 255.0

    # Convert labels to categorical (one-hot encoding)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(uniqueLabels))
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(uniqueLabels))
    x_train = np.expand_dims(x_train, axis=-1)  # (num_samples, height, width, 1)
    x_test = np.expand_dims(x_test, axis=-1)

    # Set input shape based on your image data (e.g., (height, width, channels))
    input_shape = x_train.shape[1:]  # Height, Width, Channels
    return input_shape, x_train, x_test, y_train, y_test

In [ ]:
# Define the CNN model
def cnnTraining(input_shape):
    model = models.Sequential()
    
    # First convolutional layer
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    # Second convolutional layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    # Third convolutional layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    # Flatten the output
    model.add(layers.Flatten())
    return model

In [ ]:
unique_labels = sorted(list(set(label_list)))  # Get unique classes
num_classes = len(unique_labels)

input_shape1, x_train_1, x_test_1, y_train_1, y_test_1 = lettersSplit(image_list1, label_list, unique_labels)
input_shape2, x_train_2, x_test_2, y_train_2, y_test_2 = lettersSplit(image_list2, label_list, unique_labels)
input_shape3, x_train_3, x_test_3, y_train_3, y_test_3 = lettersSplit(image_list3, label_list, unique_labels)
input_shape4, x_train_4, x_test_4, y_train_4, y_test_4 = lettersSplit(image_list4, label_list, unique_labels)
input_shape5, x_train_5, x_test_5, y_train_5, y_test_5 = lettersSplit(image_list5, label_list, unique_labels)


In [ ]:
input1 = layers.Input(shape=image_list1[0].shape)
input2 = layers.Input(shape=image_list2[0].shape)
input3 = layers.Input(shape=image_list3[0].shape)
input4 = layers.Input(shape=image_list4[0].shape)
input5 = layers.Input(shape=image_list5[0].shape)

# Create the model
cnn1 = cnnTraining(input_shape1)(input1)
cnn2 = cnnTraining(input_shape2)(input2)
cnn3 = cnnTraining(input_shape3)(input3)
cnn4 = cnnTraining(input_shape4)(input4)
cnn5 = cnnTraining(input_shape5)(input5)

x = layers.Concatenate([cnn1, cnn2, cnn3, cnn4, cnn5])
# x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(num_classes, activation='softmax')(x)

model = layers.Model(inputs=[input1, input2, input3, input4, input5], outputs = x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()